In [ ]:
!pip install nibabel matplotlib pillow torch torchvision
from google.colab import drive
drive.mount('/content/drive')

Split the nii file into slice png files and save the affine matrix

In [ ]:
import nibabel as nib
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import json  # 引入 json 用于保存 affine 矩阵

def extract_slices_with_torch(input_folder, output_folder, affine_folder):
    # List all nii.gz files in the folder
    nii_files = [f for f in os.listdir(input_folder) if f.endswith('.nii.gz')]

    for nii_file in nii_files:
        # Load the NIfTI file
        img = nib.load(os.path.join(input_folder, nii_file))
        data = img.get_fdata()
        affine_matrix = img.affine

        # Convert to PyTorch tensor
        tensor = torch.tensor(data).float()

        # Normalize slice values to [0, 255]
        tensor = 255 * (tensor - tensor.min()) / (tensor.max() - tensor.min())

        # Create a sub-folder to store the output PNGs
        sub_folder = os.path.join(output_folder, os.path.splitext(nii_file)[0])
        if not os.path.exists(sub_folder):
            os.makedirs(sub_folder)  # Make sure the directory is created

        # Convert each slice to PIL Image and save as PNG
        for i in range(tensor.shape[2]):
            slice = tensor[:, :, i]
            img = transforms.ToPILImage()(slice.byte())
            output_path = f"{sub_folder}/slice_{i:03d}.png"
            img.save(output_path)

        # Save affine matrix in JSON format
        affine_path = os.path.join(affine_folder, os.path.splitext(nii_file)[0] + '_affine.json')
        if not os.path.exists(os.path.dirname(affine_path)):
            os.makedirs(os.path.dirname(affine_path))
        with open(affine_path, 'w') as f:
            json.dump(affine_matrix.tolist(), f)

        # Output the progress
        print(f"Finished processing {nii_file} and saved affine matrix to {affine_path}")

# Example usage
input_folder = '/content/drive/My Drive/mia/mri_taskc/taskC'
output_folder = '/content/drive/My Drive/mia/mri_taskc/taskc_slices'
affine_folder = '/content/drive/My Drive/mia/mri_taskc/taskc_affine'  # 设定一个文件夹用于存储 affine 矩阵信息
extract_slices_with_torch(input_folder, output_folder, affine_folder)


Finished processing subject_59_t1w.nii.gz and saved affine matrix to /content/drive/My Drive/mia/mri_taskc/taskc_affine/subject_59_t1w.nii_affine.json
Finished processing subject_60_t1w.nii.gz and saved affine matrix to /content/drive/My Drive/mia/mri_taskc/taskc_affine/subject_60_t1w.nii_affine.json
Finished processing subject_52_t1w.nii.gz and saved affine matrix to /content/drive/My Drive/mia/mri_taskc/taskc_affine/subject_52_t1w.nii_affine.json
Finished processing subject_53_t1w.nii.gz and saved affine matrix to /content/drive/My Drive/mia/mri_taskc/taskc_affine/subject_53_t1w.nii_affine.json
Finished processing subject_61_t1w.nii.gz and saved affine matrix to /content/drive/My Drive/mia/mri_taskc/taskc_affine/subject_61_t1w.nii_affine.json


In [ ]:
import nibabel as nib
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import json

def extract_single_subject(input_folder, output_folder, affine_folder, filename):
    # Build the full path to the file
    file_path = os.path.join(input_folder, filename)

    try:
        # Load the NIfTI file
        img = nib.load(file_path)
        data = img.get_fdata()
        affine_matrix = img.affine

        # Convert to PyTorch tensor
        tensor = torch.tensor(data).float()

        # Normalize slice values to [0, 255]
        tensor = 255 * (tensor - tensor.min()) / (tensor.max() - tensor.min())

        # Create a sub-folder to store the output PNGs
        sub_folder = os.path.join(output_folder, os.path.splitext(filename)[0])
        if not os.path.exists(sub_folder):
            os.makedirs(sub_folder)  # Make sure the directory is created

        # Convert each slice to PIL Image and save as PNG
        for i in range(tensor.shape[2]):
            slice = tensor[:, :, i]
            img = transforms.ToPILImage()(slice.byte())
            output_path = f"{sub_folder}/slice_{i:03d}.png"
            img.save(output_path)

        # Save affine matrix in JSON format
        affine_path = os.path.join(affine_folder, os.path.splitext(filename)[0] + '_affine.json')
        if not os.path.exists(os.path.dirname(affine_path)):
            os.makedirs(os.path.dirname(affine_path))
        with open(affine_path, 'w') as f:
            json.dump(affine_matrix.tolist(), f)

        # Output the progress
        print(f"Finished processing {filename} and saved affine matrix to {affine_path}")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Example usage for subject 59
input_folder = '/content/drive/My Drive/mia/mri_taskc/taskC'
output_folder = '/content/drive/My Drive/mia/mri_taskc/taskc_slices'
affine_folder = '/content/drive/My Drive/mia/mri_taskc/taskc_affine'
filename = 'subject_59_t1w.nii.gz'
extract_single_subject(input_folder, output_folder, affine_folder, filename)


Finished processing subject_59_t1w.nii.gz and saved affine matrix to /content/drive/My Drive/mia/mri_taskc/taskc_affine/subject_59_t1w.nii_affine.json


Rotate the image to the correct orientation according to the actual situation

In [ ]:
from PIL import Image
import os
import re


def rotate_images_in_directory(input_dir, output_dir, rotation_degree=90, start_number=0):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # 创建输出目录如果它不存在

    # 初始化编号
    number = start_number

    # 遍历指定目录
    for filename in sorted(os.listdir(input_dir)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # 确保文件是图片
            file_path = os.path.join(input_dir, filename)
            # 打开并旋转图片
            with Image.open(file_path) as img:
                rotated_img = img.rotate(-rotation_degree, expand=True)  # 旋转90度
                # 生成新的文件名
                new_filename = re.sub(r'\d+', lambda x: f"{number:03d}", filename)
                # 保存到输出目录
                rotated_img.save(os.path.join(output_dir, new_filename))
                print(f'Rotated {filename} and saved as {new_filename} to {output_dir}')
                number += 1  # 增加编号



# 用法示例
input_directory = '/content/drive/My Drive/mia/mri_taskc/taskc_slices/subject_61_t1w.nii'
output_directory = '/content/drive/My Drive/mia/mri_taskc/taskc_slices2/subject_61_t1w.nii'
rotate_images_in_directory(input_directory, output_directory)

In [ ]:
from PIL import Image
import os
import re


def rotate_images_in_directory(input_dir, output_dir, rotation_degree=-90, start_number=0):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # 创建输出目录如果它不存在

    # 初始化编号
    number = start_number

    # 遍历指定目录
    for filename in sorted(os.listdir(input_dir)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # 确保文件是图片
            file_path = os.path.join(input_dir, filename)
            # 打开并旋转图片
            with Image.open(file_path) as img:
                rotated_img = img.rotate(-rotation_degree, expand=True)  # 旋转90度
                # 生成新的文件名
                new_filename = re.sub(r'\d+', lambda x: f"{number:03d}", filename)
                # 保存到输出目录
                rotated_img.save(os.path.join(output_dir, new_filename))
                print(f'Rotated {filename} and saved as {new_filename} to {output_dir}')
                number += 1  # 增加编号



# 用法示例
input_directory = '/content/drive/My Drive/mia/mri_taskc3/taskc_slices2/subject_53_t1w.nii'
output_directory = '/content/drive/My Drive/mia/mri_taskc3/taskc_slices3/subject_53_t1w.nii'
rotate_images_in_directory(input_directory, output_directory)

Crop image size as needed

In [ ]:
import os
from PIL import Image

def resize_images_to_fixed_size(directory, target_size=(256,176)):
    # 遍历文件夹中的PNG图像并调整其尺寸
    images = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
    for img_path in images:
        img = Image.open(img_path)
        print(f"原始图像 {img_path} 的尺寸为: {img.size}")
        if img.size != target_size:
            img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
            img_resized.save(img_path)  # 覆盖原图
            print(f"调整后的图像 {img_path} 的尺寸为: {target_size}")
        else:
            print(f"图像 {img_path} 的尺寸已经是 {target_size}")

# 文件夹路径
directory = '/content/drive/My Drive/mia/mri_taskc3/taskc_slices3/subject_61_t1w.nii'

# 调整指定文件夹中的图像尺寸
print(f"正在处理文件夹: {directory}")
resize_images_to_fixed_size(directory)

In [ ]:
import os
from PIL import Image

def resize_images_to_fixed_size(directory, target_size=(150, 256)):
    # 遍历文件夹中的PNG图像并调整其尺寸
    images = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
    for img_path in images:
        img = Image.open(img_path)
        print(f"原始图像 {img_path} 的尺寸为: {img.size}")
        if img.size != target_size:
            img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
            img_resized.save(img_path)  # 覆盖原图
            print(f"调整后的图像 {img_path} 的尺寸为: {target_size}")
        else:
            print(f"图像 {img_path} 的尺寸已经是 {target_size}")

# 文件夹路径
directory = '/content/drive/My Drive/mia/output3'

# 调整指定文件夹中的图像尺寸
print(f"正在处理文件夹: {directory}")
resize_images_to_fixed_size(directory)

Read the size of an image file

In [ ]:
import os
from PIL import Image

def print_image_sizes(directory):
    # 遍历文件夹中的PNG图像并打印其尺寸
    images = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
    for img_path in images:
        img = Image.open(img_path)
        print(f"图像 {img_path} 的尺寸为: {img.size}")

# 文件夹路径
directory = '/content/drive/My Drive/mia/mri_taskc3/taskc_slices3/subject_59_t1w.nii'

# 打印指定文件夹中的图像尺寸
print(f"正在处理文件夹: {directory}")
print_image_sizes(directory)


Combine the png files of the processed slice with the affine matrix to form a new nii file.

In [ ]:
import os
import json
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
import nibabel as nib

def combine_slices_to_nii(base_input_folder, base_output_folder, affine_folder):
    # Ensure the base output folder exists
    if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

    # Process each subject folder in the base input folder
    for subject_folder in os.listdir(base_input_folder):
        subject_folder_path = os.path.join(base_input_folder, subject_folder)
        if not os.path.isdir(subject_folder_path):
            continue

        print(f"Processing {subject_folder}")

        # List all PNG files in the subject folder, assuming they are named 'slice_XXX.png'
        files = sorted([os.path.join(subject_folder_path, f) for f in os.listdir(subject_folder_path) if f.endswith('.png')],
                       key=lambda x: int(x.split('_')[-1].split('.')[0]))

        if not files:
            print(f"No PNG files found in {subject_folder_path}. Please check the directory and try again.")
            continue

        # Create an empty list to store image data arrays
        slice_data = []

        for file in files:
            img = Image.open(file)
            img = img.convert('L')  # Ensure image is in grayscale
            slice_array = np.array(img)
            slice_data.append(slice_array)

        # Stack all slices along the third dimension (axis=2) to form a volume
        volume_data = np.stack(slice_data, axis=2)

        # Load the corresponding affine matrix
        affine_file_name = f"{subject_folder}_affine.json"
        affine_path = os.path.join(affine_folder, affine_file_name)
        try:
            with open(affine_path, 'r') as f:
                affine_matrix = np.array(json.load(f))
        except Exception as e:
            print(f"Error loading affine matrix for {subject_folder}: {e}, using identity matrix.")
            affine_matrix = np.eye(4)

        # Create a NIfTI image from the volume data
        new_img = nib.Nifti1Image(volume_data, affine=affine_matrix)
        output_file_path = os.path.join(base_output_folder, f"{subject_folder}.nii.gz")
        nib.save(new_img, output_file_path)

        print(f"Saved NIfTI file to {output_file_path} with custom affine matrix.")

# Example usage
base_input_folder = '/content/drive/My Drive/mia/mri_taskc3/taskc_slices3'
base_output_folder ='/content/drive/My Drive/mia/mri_taskc3/newnii'
affine_folder = '/content/drive/My Drive/mia/mri_taskc/taskc_affine'
combine_slices_to_nii(base_input_folder, base_output_folder, affine_folder)


Processing subject_61_t1w.nii
Saved NIfTI file to /content/drive/My Drive/mia/mri_taskc3/newnii/subject_61_t1w.nii.nii.gz with custom affine matrix.
Processing subject_60_t1w.nii
Saved NIfTI file to /content/drive/My Drive/mia/mri_taskc3/newnii/subject_60_t1w.nii.nii.gz with custom affine matrix.
Processing subject_59_t1w.nii
Saved NIfTI file to /content/drive/My Drive/mia/mri_taskc3/newnii/subject_59_t1w.nii.nii.gz with custom affine matrix.
Processing subject_52_t1w.nii
Saved NIfTI file to /content/drive/My Drive/mia/mri_taskc3/newnii/subject_52_t1w.nii.nii.gz with custom affine matrix.
Processing subject_53_t1w.nii
Saved NIfTI file to /content/drive/My Drive/mia/mri_taskc3/newnii/subject_53_t1w.nii.nii.gz with custom affine matrix.
